In [2]:
import numpy as np
import pandas as pd

In [3]:
def gen_learner_history(now,times,correct,alpha=0.0,delta=0.0,windows=[1/24,1,7,30]):
    times = np.array(times)
    correct = np.array(correct)
    elapse = now-times
    windows = [0.0] + [864008.0*x for x in windows] + [float('inf')]
    lh = {}
    for i in range(len(windows)-1):
        upper = windows[i+1]
        lower = windows[i]
        in_win = np.logical_and(elapse > lower,elapse <= upper)
        c_in_win = in_win * correct
        
        lh['n_w'+str(i)] = in_win.sum()
        lh['c_w'+str(i)] = c_in_win.sum()
    lh["alpha"] = alpha
    lh["delta"] = delta
    return lh

def day(x):
    return x * 864008.0
def hour(x):
    return x * 3600



In [4]:
#---Fit with torch---
#phi tensor([-0.1015,  0.1419,  0.1439,  0.1933,  0.0000])
#psi tensor([-0.1941,  0.1973,  0.0245,  0.0347,  0.0000])

#---Fit with glmer (in R)----
#    ltc_0      lop_0      ltc_1      lop_1      ltc_2      lop_2      ltc_3      lop_3      ltc_4  lop_4  
# 0.038408  -0.011849   0.048038  -0.032408   0.033345  -0.024711  -0.003275   0.005272  -0.043829  0.033895  
    
 
window_profiles = [{
    'scale' : 1/24,
    'phi' : .038408,
    'psi' : -0.011849
},
{
    'scale' : 1,
    'phi' : 0.048038,
    'psi' : -0.032408
},
{
    'scale' : 7,
    'phi' : 0.033345,
    'psi' : -0.024711
},
{
    'scale' : 30,
    'phi' : -0.003275,
    'psi' : 0.005272
},
]

def sigmoid(x):
    return 1/(1 + np.exp(-x)) 

def DASH_activation(lh,wps):
    s = 0
    for i,wp in enumerate(wps):
        s += wp["phi"]*np.log(1+lh["c_w"+str(i)])-wp["psi"]*np.log(1+lh["n_w"+str(i)])
    return lh['alpha'] - lh['delta'] + s

def DASH(lh,wps):
    x = DASH_activation(lh,wps)
    return 1/(1 + np.exp(-x)) 


In [5]:
learner_history = gen_learner_history(day(2)+hour(1),[
        day(0)+hour(0)+0,
        day(0)+hour(0)+30,
        day(0)+hour(0)+40,
        day(1)+hour(3)+0,
        day(1)+hour(3)+30,
        day(1)+hour(3)+40,
        ],[0,0,1,0,1,1],alpha=.05,delta=.3)
print(learner_history)
print(DASH(learner_history,window_profiles))

{'n_w0': 0, 'c_w0': 0, 'n_w1': 3, 'c_w1': 2, 'n_w2': 3, 'c_w2': 1, 'n_w3': 0, 'c_w3': 0, 'n_w4': 0, 'c_w4': 0, 'alpha': 0.05, 'delta': 0.3}
0.4762857747894385
